# Do the following steps to use tensorboard in notebook

### <span style='color:cyan'>do not install tb-nightly</span>

### <span style='color:cyan'>be careful [ .to(device) ] as tensorboard only accept one. Not Both </span>

## ctrl + shift + p to open command palatte in vscode and type tensorboard. Then, select the cwd.

In [24]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_size = 784 #28x28 pixels
hidden_size = 500  # can try differnet size
num_classes = 10 # classification of '0 to 9'
num_epochs = 2
batch_size = 100 
learning_rate = 0.01

train_dataset = torchvision.datasets.MNIST(root='./feed_forward_net',train = True,transform=transforms.ToTensor(),download = True)
test_dataset = torchvision.datasets.MNIST(root='./feed_forward_net',train = False,transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,batch_size=batch_size,shuffle=False)

In [25]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        # no activation and no softmax at the end
        return out

In [26]:
model =NeuralNet(input_size,hidden_size,num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [27]:
examples = iter(train_loader)
samples, labels = next(examples)

# have to be [.to(device)]
writer.add_graph(model,samples.reshape(-1,28*28).to(device))

In [28]:
n_total_steps = len(train_loader)
running_loss = 0.0
running_correct = 0.0
for epoch in range(num_epochs):
    for i, (images,labels)  in enumerate(train_loader):
        # input size is 28x28 = 784 
        # origin shape: [100, 1, 28, 28]
        # resized: [100, 784]
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs,labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predictions = torch.max(outputs.data,1)
        running_correct += (predictions==labels).sum().item()

        # for every 100 steps we print the following.
        if (i+1)%100 == 0:
            print(f'epoch {epoch+1}/{num_epochs},step{i+1}/{n_total_steps},loss = {loss.item():.3f}')
            # we write the training loss for every 100 steps.
            writer.add_scalar('training loss', running_loss/100, epoch*n_total_steps+i)
            writer.add_scalar('training accuracy', running_correct/100, epoch*n_total_steps+i)
            running_loss = 0.0
            running_correct = 0.0

epoch 1/2,step100/600,loss = 0.276
epoch 1/2,step200/600,loss = 0.155
epoch 1/2,step300/600,loss = 0.256
epoch 1/2,step400/600,loss = 0.202
epoch 1/2,step500/600,loss = 0.063
epoch 1/2,step600/600,loss = 0.161
epoch 2/2,step100/600,loss = 0.065
epoch 2/2,step200/600,loss = 0.084
epoch 2/2,step300/600,loss = 0.189
epoch 2/2,step400/600,loss = 0.093
epoch 2/2,step500/600,loss = 0.182
epoch 2/2,step600/600,loss = 0.149


In [29]:
labs = []
preds = []
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        # value, index will return
        _, predictions = torch.max(outputs.data,1)
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()
        
        class_prediction = [nn.functional.softmax(output,dim = 0) for output in outputs]
        preds.append(class_prediction)    
        labs.append(predictions)
    labs = torch.cat(labs).to(device)
    preds = torch.cat([torch.stack(batch) for batch in preds]).to(device)
    
    
    acc = 100.0 * n_correct/n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')
    classes = range(10)
    for i in classes: 
        labs_i = labs == i
        preds_i = preds[:,i]
        writer.add_pr_curve(str(i), labs_i, preds_i, global_step=0)
    writer.close()

Accuracy of the network on the 10000 test images: 96.57 %
